In [5]:
# Install dependencies (run once)
%pip install streamlit pandas openpyxl pyngrok python-dotenv

In [6]:
# Create the Streamlit app file - Dynamic Path
from pathlib import Path

# Use current working directory or notebook directory
APP_DIR = Path.cwd()
APP_PATH = APP_DIR / "guardian_emails_app.py"

print(f"Creating app at: {APP_PATH}")

app_code = r'''\
# -*- coding: utf-8 -*-
"""
guardian_emails_app.py
----------------------
Streamlit UI for extracting guardian email addresses from class lists.
"""
import io
import re
from typing import List, Sequence

import pandas as pd
import streamlit as st

st.set_page_config(page_title="Adressutdrag – vårdnadshavare", layout="wide")

st.title("Adressutdrag: vårdnadshavare från klasslista")
st.caption("Ladda upp en CSV/XLSX eller klistra in CSV-text. Data stannar i sessionen.")

EMAIL_RE = re.compile(r'([A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,15})')
GUARDIAN_HINTS = ["vårdnad", "vardnad", "förälder", "foralder", "parent", "guardian", "caregiver"]
EMAIL_HINTS = ["e-post", "epost", "email", "e-mail", "mejl", "mail"]

@st.cache_data(show_spinner=False)
def read_any_file(uploaded_file) -> pd.DataFrame:
    """Read CSV or XLSX from Streamlit UploadedFile."""
    name = uploaded_file.name.lower()
    if name.endswith((".xlsx", ".xls")):
        df = pd.read_excel(uploaded_file, dtype=str).fillna("")
        return df
    content = uploaded_file.getvalue()
    try:
        text = content.decode("utf-8-sig")
    except UnicodeDecodeError:
        text = content.decode("utf-8", errors="replace")
    buf = io.StringIO(text)
    df = pd.read_csv(buf, sep=None, engine="python", dtype=str).fillna("")
    return df

def read_from_text(text: str) -> pd.DataFrame:
    """Read DataFrame from pasted CSV text."""
    if not text or not text.strip():
        raise ValueError("No text pasted.")
    buf = io.StringIO(text)
    df = pd.read_csv(buf, sep=None, engine="python", dtype=str).fillna("")
    return df

def prioritized_columns(columns: Sequence[str]) -> List[str]:
    cols = list(columns)
    lower = [c.lower() for c in cols]
    both = [
        cols[i]
        for i, c in enumerate(lower)
        if any(h in c for h in GUARDIAN_HINTS) and any(h in c for h in EMAIL_HINTS)
    ]
    if both:
        return both
    email_cols = [cols[i] for i, c in enumerate(lower) if any(h in c for h in EMAIL_HINTS)]
    if email_cols:
        return email_cols
    guardian_cols = [cols[i] for i, c in enumerate(lower) if any(h in c for h in GUARDIAN_HINTS)]
    if guardian_cols:
        return guardian_cols
    return cols

def harvest_emails_from_cells(cells: Sequence[str]):
    emails = []
    for cell in cells:
        if not cell:
            continue
        found = EMAIL_RE.findall(cell)
        if found:
            emails.extend(found)
    return sorted(set(e.lower() for e in emails))

def extract_emails(df: pd.DataFrame):
    if df.empty or not list(df.columns):
        return []
    cols = prioritized_columns(df.columns)
    cells = df[cols].astype(str).values.ravel().tolist()
    emails = harvest_emails_from_cells(cells)
    if not emails:
        cells_all = df.astype(str).values.ravel().tolist()
        emails = harvest_emails_from_cells(cells_all)
    return emails

with st.sidebar:
    st.header("Inläsning")
    uploaded = st.file_uploader("Ladda upp CSV eller XLSX", type=["csv", "xlsx", "xls"])
    st.markdown("**eller**")
    pasted = st.text_area("Klistra in CSV-text här", height=160, placeholder="Förnamn,Efternamn,...")
    st.divider()
    sep = st.text_input("Separator i utdata", value=";")
    preview = st.checkbox("Visa förhandsgranskning av tabellen", value=False)
    run = st.button("Extrahera adresser", type="primary")

def render_df_preview(df: pd.DataFrame, max_rows: int = 50):
    st.subheader("Förhandsgranskning")
    if len(df) > max_rows:
        st.info(f"Visar de första {max_rows} raderna av totalt {len(df)}.")
    st.dataframe(df.head(max_rows), use_container_width=True)

def process(uploaded, pasted, sep: str):
    if uploaded is None and not pasted.strip():
        st.warning("Ladda upp en fil eller klistra in CSV-text.")
        return

    try:
        if uploaded is not None:
            df = read_any_file(uploaded)
        else:
            df = read_from_text(pasted)
    except Exception:
        st.error("Klistra in en giltig klasslista")
        return

    if preview:
        render_df_preview(df)

    emails = extract_emails(df)
    if not emails:
        st.error(
            "Inga mejladresser hittades. "
            "Kontrollera att listan innehåller kolumner med vårdnadshavares e-post."
        )
        return

    out_str = (sep or ";").join(emails)
    st.success(f"Hittade {len(emails)} unika adresser.")
    st.subheader("Semikolonseparerad sträng")
    st.code(out_str, language=None)

    st.download_button(
        label="Ladda ned som .txt",
        data=out_str,
        file_name="vårdnadshavare_adresser.txt",
        mime="text/plain",
    )

    st.caption("Tips: Kopiera från rutan ovan (copy-ikonen) och klistra in i Outlook Bcc.")

if run:
    process(uploaded, pasted, sep)
'''

APP_PATH.write_text(app_code, encoding="utf-8")
print(f"Wrote app to: {APP_PATH}")

Creating app at: /content/guardian_emails_app.py
Wrote app to: /content/guardian_emails_app.py


In [ ]:
# Load secrets (no hardcoded tokens)
import os
from getpass import getpass
from pathlib import Path
from dotenv import load_dotenv

# 1. Try to find .env file
def find_env_file():
    """Search for .env file starting from current directory up to root"""
    search_paths = [Path.cwd(), Path(__file__).parent if "__file__" in locals() else None]
    start_paths = [p for p in search_paths if p]
    checked = set()
    
    for start_dir in start_paths:
        current = start_dir.resolve()
        for _ in range(6):
            if str(current) in checked: break
            checked.add(str(current))
            env_path = current / ".env"
            if env_path.exists(): return env_path
            if current.parent == current: break
            current = current.parent
    return None

print(f"Current Working Directory: {Path.cwd()}")

ENV_PATH = find_env_file()
if ENV_PATH:
    print(f"✅ Found .env at: {ENV_PATH}")
    load_dotenv(ENV_PATH, override=True)
else:
    print("❌ WARNING: Could not find .env file in parent directories.")

# 2. Load token from env or prompt
token = os.environ.get("NGROK_AUTHTOKEN")
if not token:
    token = getpass("Enter NGROK_AUTHTOKEN (will not be echoed): ").strip()
    if token:
        os.environ["NGROK_AUTHTOKEN"] = token

print(f"NGROK_AUTHTOKEN loaded: {'✅ Yes' if token else '❌ No'}")
if not token:
    raise ValueError("NGROK_AUTHTOKEN is missing! Please set it in the environment.")

In [ ]:
# Start Streamlit + ngrok tunnel - Dynamic Paths
import os
import subprocess
import time
from pathlib import Path

from pyngrok import conf, ngrok

# Use current working directory
APP_DIR = Path.cwd()
APP_PATH = APP_DIR / "guardian_emails_app.py"

# Kill any existing ngrok tunnels
ngrok.kill()
time.sleep(2)

PORT = 8501

print(f"Starting Streamlit app from: {APP_PATH}")

# 1) Start Streamlit in background
if not APP_PATH.exists():
    raise FileNotFoundError(f"App file not found at {APP_PATH}. Did you run the previous cell?")

streamlit_proc = subprocess.Popen(
    [
        "streamlit", "run", str(APP_PATH),
        "--server.port", str(PORT),
        "--server.headless", "true",
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)
time.sleep(3)

# 2) Set ngrok token from environment (loaded in previous cell)
NGROK_TOKEN = os.environ.get("NGROK_AUTHTOKEN", "")
if NGROK_TOKEN:
    print("Setting ngrok auth token...")
    ngrok.set_auth_token(NGROK_TOKEN)
else:
    print("WARNING: NGROK_AUTHTOKEN not set in environment variables")

# 3) Configure EU region for lower latency
cfg = conf.get_default()
cfg.region = "eu"
conf.set_default(cfg)

# 4) Optional Basic Auth
basic_user = os.environ.get("BASIC_USER", "")
basic_pass = os.environ.get("BASIC_PASS", "")
auth = f"{basic_user}:{basic_pass}" if basic_user and basic_pass else None

# 5) Create HTTPS tunnel
try:
    tunnel = ngrok.connect(addr=PORT, proto="http", bind_tls=True, auth=auth)
    public_url = tunnel.public_url

    print(f"\n{'='*60}")
    print(f"Streamlit app running at: {public_url}")
    print(f"{'='*60}\n")
    if auth:
        print("NOTE: Tunnel is password-protected with Basic Auth.")
except Exception as e:
    print(f"ERROR: Failed to start ngrok tunnel: {e}")
    # Print helpful debugging info
    print(f"NGROK_AUTHTOKEN present: {'Yes' if NGROK_TOKEN else 'No'}")